In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('paper')
sns.set_style('white')

In [3]:
import pandas as pd
import configs
import numpy as np

In [4]:
import os
import json

## Create setup dicts
1. Word Unit+Labels: `morph, token, multitok`
1. Char Arch: `char_lstm, char_cnn, no_char`
1. Word Embedding: `ft_tok, ft_yap, ft_tok_oov, ft_yap_oov, w2v_tok, w2v_yap, no_word_embed`

3 x 3 x 7 = **63 dicts**

## Create PER-LOC-ORG only datasets

In [5]:
data_folder = '../NER/data/for_ncrf'

new_datasets = {
    'morph': {
        '_unit': 'morpheme',
        '_scheme': 'bioes',
        'seg': False,
        'train_dir': 'morph_gold_train_plo.bmes',
        'dev_dir': 'morph_gold_dev_plo.bmes',
        'test_dir': 'morph_gold_test_plo.bmes', 
    },
    'token': {
        '_unit': 'token',
        '_scheme': 'bioes',
        'seg': False,
        'train_dir': 'token_gold_train_fix_plo.bmes',
        'dev_dir': 'token_gold_dev_fix_plo.bmes',
        'test_dir': 'token_gold_test_fix_plo.bmes',
    },
}

In [6]:
default_grid = { 
        # FIXED
        'word_seq_feature': 'LSTM',
        'word_emb_dim': 300,
        'char_emb_dim': 30,
        'iteration': 200,
        'bilstm': True,
        'norm_word_emb': False,
        'norm_char_emb': False,
        'ave_batch_loss': False,
        'use_crf': True,
        'l2': 1e-8,
        'lstm_layer': 2,
        'batch_size': 8,
        'number_normalized': True,
        'optimizer': 'SGD',
        'lr_decay': 0.05,
        'momentum': 0,
        'nbest': 1,
        'hidden_dim': 200,
        'dropout': 0.5,

    }
    
dataset_grids = {
    'multitok': {
        'learning_rate': 0.005,
    },
    'morph': {
        'learning_rate': 0.01,
    },
    'token': {
        'learning_rate': 0.01,
    },
}
arch_grids = {
    'char_lstm': {
        'char_seq_feature': 'LSTM',
        'use_char': True,
        'char_hidden_dim': 70, 
    },
    'char_cnn': {
        'char_seq_feature': 'CNN',
        'use_char': True,
        'char_hidden_dim': 70,
        'char_kernel_size': 7,
    },
    'no_char': {
        'use_char': False,
     },
}


In [7]:
word_embedding_files = {
    #'ft_yap': '../wordembedding-hebrew/vectors_alt_tok/wikipedia.alt_tok.yap_form.fasttext_skipgram.model.vec.nofirstline',
    #'ft_tok': '../wordembedding-hebrew/vectors_alt_tok/wikipedia.alt_tok.tokenized.fasttext_skipgram.model.vec.nofirstline',
    'ft_oov_yap': 'data/htb_all_words.wikipedia.alt_tok.yap_form.fasttext_skipgram.txt',
    'ft_oov_tok': 'data/htb_all_words.wikipedia.alt_tok.tokenized.fasttext_skipgram.txt',
    #'w2v_yap': '../wordembedding-hebrew/vectors_alt_tok/wikipedia.alt_tok.yap_form.word2vec_skipgram.txt.nofirstline',
    #'w2v_tok': '../wordembedding-hebrew/vectors_alt_tok/wikipedia.alt_tok.tokenized.word2vec_skipgram.txt.nofirstline',
    #'glv_yap': '../wordembedding-hebrew/vectors_alt_tok/wikipedia.alt_tok.yap_form.glove.txt',
    #'glv_tok': '../wordembedding-hebrew/vectors_alt_tok/wikipedia.alt_tok.tokenized.glove.txt',
    #'no_word': None,
}

In [8]:
models_folder = 'final_setup/noseg_plo_models'
conf_folder =   'final_setup/noseg_plo_conf'
json_folder =   'final_setup/noseg_plo_conf_json'
logs_folder =   'final_setup/noseg_plo_logs'

In [9]:
seed_num_options = np.arange(44, 54)
seed_num_options

array([44, 45, 46, 47, 48, 49, 50, 51, 52, 53])

In [10]:
def create_conf_dict(model_base_name, dataset, arch, emb_name, seed_num):
    full_conf_dict = {}
    full_conf_dict['status'] = 'train'
    full_conf_dict['model_dir'] = os.path.join(models_folder, model_base_name)
    
    for k, v in new_datasets[dataset].items():
        if not k.startswith('_'):
            if k in ['train_dir', 'dev_dir', 'test_dir']:
                full_conf_dict[k] = os.path.join(data_folder, v)
            else:
                full_conf_dict[k] = v
    
    if not(emb_name == 'no_word' or word_embedding_files[emb_name] is None):
        full_conf_dict['word_emb_dir'] = word_embedding_files[emb_name]
    
    
    full_conf_dict.update(default_grid)
    full_conf_dict.update(dataset_grids[dataset])
    full_conf_dict.update(arch_grids[arch])
            
    return full_conf_dict
    

In [11]:
ds_embeds = {'morph': ['ft_oov_yap', 'ft_oov_tok'], 
             'token': ['ft_oov_tok'],}
ds_archs = {'morph': ['char_cnn'],
            'token': ['char_cnn'],}

confs = {}
for dataset in new_datasets:
    for arch in ds_archs[dataset]:
        for emb_name in ds_embeds[dataset]:
            for seed_num in seed_num_options:
                model_base_name = '.'.join([dataset, arch, emb_name, str(seed_num)+'_seed'])
                confs[model_base_name] = create_conf_dict(model_base_name, dataset, 
                                                          arch, emb_name, seed_num)
            

In [12]:
len(confs)

30

In [13]:
import pickle
pickle.dump(confs, open('final_setup/noseg_plo_confs.pkl', 'wb'))

## Create conf files for setup dicts
1. Random Seed: 10 different `(44, 45, 46...)`
1. `morph.charlstm.ft_tok.44_seed.conf`
1. `multitok.nochar.no_word_embed.47_seed.conf`

63 * 10 = **630 conf files**

In [14]:
if not os.path.exists(models_folder):
    os.mkdir(models_folder)
if not os.path.exists(conf_folder):
    os.mkdir(conf_folder)
if not os.path.exists(json_folder):
    os.mkdir(json_folder)
if not os.path.exists(logs_folder):
    os.mkdir(logs_folder)

In [15]:
for name, conf in confs.items():
    conf_path = os.path.join(conf_folder, name+'.conf')
    with open(conf_path, 'w', encoding='utf8') as of:
        for k, v in conf.items():
            of.write(k+'='+str(v)+'\n')
    json_path = os.path.join(json_folder, name+'.json')
    with open(json_path, 'w') as of:
        of.write(json.dumps(conf))

## Create `main.X.py` files
Only difference is `seed_num`: 
- `main.44.py` will have `seed_num = 44`

## Create `final_setup_run.py`
1. seed_num match: runs `.conf` files with matching `main.X.py` file **only**.
1. Choose device. 
1. Choose conf prefix.
1. Skip confs that are running or ran already (using `.dset` file)

In [7]:
emb_options = list(word_embedding_files.keys())+[None]
emb_options

['alt_tok_yap_ft_sg',
 'alt_tok_tokenized_ft_sg',
 'htb_all_alt_tok_yap_ft_sg',
 'htb_all_alt_tok_tokenized_ft_sg',
 'alt_tok_yap_w2v_sg',
 'alt_tok_tokenized_w2v_sg',
 None]

## Read logs

In [50]:
import pickle
confs = pickle.load( open('final_setup/noseg_plo_confs.pkl', 'rb'))

In [293]:
import re
import os
DEV_RES_LINE = re.compile('Dev: .*; acc: (?P<acc>[^,]+)(?:, p: (?P<p>[^,]+), r: (?P<r>[^,]+), f: (?P<f>[-\d\.]+))?')
#Dev: time: 0.94s speed: 536.09st/s; acc: 0.9043
#Dev: time: 3.42s, speed: 146.59st/s; acc: 0.9546, p: 0.7577, r: 0.6393, f: 0.6935

mtimes = []
res = []
archs = []
for f in os.scandir(logs_folder):
    if f.name.startswith('.ipy'):
        continue
    mtimes.append(os.path.getmtime(f.path))
    model_base_name = '.'.join(f.name.split('.')[:-1])
    model_no_seed = '.'.join(f.name.split('.')[:-2])
    unit, arch, w_embed, seed_num = f.name.split('.')[:-1]
    archs.append(arch)
    matching_conf = confs[model_base_name]
    params = { 'model_base_name': model_base_name, 'arch': arch, 
              'unit': unit, 'w_embed': w_embed, 'seed_num': seed_num,
              'model_no_seed': model_no_seed,}
    params.update(matching_conf)
    with open(f.path, 'r') as fp:
        i= 0
        for line in fp:
            m = DEV_RES_LINE.match(line)
            if m:
                r = m.groupdict().copy()
                for k, v in r.items():
                    if v is not None:
                        r[k] = float(v)
                r.update(params)
                r['epoch'] = i
                i+=1
                res.append(r)

rdf = pd.DataFrame(res)

rdf['model_file_name'] = rdf.model_base_name + '.' + rdf.epoch.astype(str) + '.model'
rdf['dset_file_name'] =  rdf.model_base_name +'.dset'
rdf['char_seq_feature'] = rdf.char_seq_feature.fillna('NoChar')

rdf['relevant_score'] = rdf.f.fillna(rdf.acc)

def get_embed_unit(s):
    if 'yap' in s:
        return 'morph'
    elif 'tok' in s:
        return 'token'
    return 'na'

def get_clash_match(s):
    if s.embed_unit=='na':
        return 'na'
    elif s.embed_unit==s.input_unit:
        return 'Match'
    else:
        return 'Clash'
    
rdf['input_unit'] = rdf.unit.apply(lambda x: 'morph' if x=='morph' else 'token')
rdf['embed_unit'] = rdf.w_embed.apply(get_embed_unit)
rdf['embed_type'] = rdf.w_embed.str.replace('_tok|_yap', '')
rdf['cm'] = rdf.apply(get_clash_match, axis=1)

erdf = rdf[(rdf.groupby(['seed_num', 'arch', 'unit', 'w_embed']).relevant_score.transform(max)==rdf.relevant_score) ]
erdf = erdf[(erdf.groupby(['seed_num', 'arch', 'unit', 'w_embed']).epoch.transform(min)==erdf.epoch) ]

In [294]:
erdf.shape

(30, 49)

In [295]:
erdf.groupby(['unit', 'arch', 'w_embed']).seed_num.nunique().unstack()

,w_embed,ft_oov_tok,ft_oov_yap
unit,arch,,
morph,char_cnn,10.0,10.0
token,char_cnn,10.0,NaN


In [296]:
print ('Mean time per run:', round((max(mtimes) - min(mtimes) )/ len(mtimes) / 60, 2), 'minutes')

Mean time per run: 54.25 minutes


In [297]:
erdf.groupby(['unit', 'arch', 'embed_type', 'cm']).relevant_score.mean().unstack([-2,-1]).mul(100).round(2)

embed_type     ft_oov       
cm              Clash  Match
unit  arch                  
morph char_cnn  97.02  97.11
token char_cnn    NaN  96.81

In [298]:
import numpy as np 
def perc(n):
    def perc_(x):
        return np.percentile(x, n)
    perc_.__name__ = 'perc_%s' % n
    return perc_

In [299]:
erdf.groupby(['unit', 'char_seq_feature']).relevant_score.agg(['max', 'min', 'mean', 'std', 'median', perc(95)]).mul(100).round(2)

,,max,min,mean,std,median,perc_95
unit,char_seq_feature,,,,,,
morph,CNN,97.22,96.93,97.06,0.09,97.06,97.21
token,CNN,96.93,96.66,96.81,0.09,96.82,96.90


In [300]:
erdf.groupby(['unit', 'char_seq_feature']).relevant_score.agg(['max', 'min', 'mean', 'std', 'median', perc(95)]).mul(100).round(2)

,,max,min,mean,std,median,perc_95
unit,char_seq_feature,,,,,,
morph,CNN,97.22,96.93,97.06,0.09,97.06,97.21
token,CNN,96.93,96.66,96.81,0.09,96.82,96.90


In [301]:
erdf.to_pickle('final_setup/noseg_plo_erdf.pkl')

## Decode

In [302]:
output_folder = 'final_setup/noseg_plo_decode_output'
decode_conf_folder = 'final_setup/noseg_plo_decode_conf'

In [303]:
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

if not os.path.exists(decode_conf_folder):
    os.mkdir(decode_conf_folder)
    
decode_sets = {
    'morph': {
        'morph_dev_gold': '../NER/data/for_ncrf/morph_gold_dev_plo.bmes',
        'morph_dev_yap': '../NER/data/for_ncrf/morph_yap_dev_dummy_o.bmes',
        'morph_test_gold': '../NER/data/for_ncrf/morph_gold_test_plo.bmes',
        'morph_test_yap': '../NER/data/for_ncrf/morph_yap_test_dummy_o.bmes',
    },
    'token': {
        'token_dev': '../NER/data/for_ncrf/token_gold_dev_fix_plo.bmes',
        'token_test': '../NER/data/for_ncrf/token_gold_test_fix_plo.bmes',
    },
    'multitok': {
        'token_dev': '../NER/data/for_ncrf/token_gold_dev_concat_plo.bmes',
        'token_test': '../NER/data/for_ncrf/token_gold_test_concat_plo.bmes',
    }
}

In [304]:
params = { 'status': 'decode' }

for i, row in erdf.iterrows():
    unit = row['unit']
    for name, set_path in decode_sets[unit].items():
        row_par = params.copy()
        row_par['load_model_dir'] = os.path.join(models_folder, row['model_file_name'])
        row_par['dset_dir'] = os.path.join(models_folder, row['dset_file_name'])
        row_par['decode_dir'] = os.path.join(output_folder, name+'.'+row['model_base_name']+'.bmes')
        row_par['raw_dir'] = set_path
        
        conf_path = os.path.join(decode_conf_folder, name+'.'+row['model_base_name']+'.decode.conf')
        if not os.path.exists(conf_path):
            with open(conf_path, 'w', encoding='utf8') as of:
                for k, v in row_par.items():
                    of.write(k+'='+str(v)+'\n')        
 

In [305]:
import os, re

In [274]:
pred_line = re.compile('Predict raw 1-best result has been written into file.*')
bads = []
for f in os.scandir('final_setup/noseg_plo_decode_logs'):
    if f.name=='.ipynb_checkpoints' or f.name=='.log':
        continue
    with open(f.path, 'r') as fp:
        data = fp.read()
        if len(re.findall(pred_line, data))==0:
            bads.append (f.name)
            #os.remove(f.path)
sorted(bads)

[]

In [275]:
from collections import Counter

In [276]:
xxx = []
for f in os.scandir('final_setup/noseg_plo_decode_output'):
    if f.name=='.ipynb_checkpoints' or f.name=='.bmes':
        continue
    elif 'pruned' in f.name:
        xxx.append('.'.join(f.name.split('.')[:-2]))
Counter(xxx).most_common()

[]

## Evaluate decoded folder

In [277]:
erdf = pd.read_pickle('final_setup/noseg_plo_erdf.pkl')

In [278]:
import sys
sys.path.append('../NER')
import ne_evaluate_mentions as nem

In [306]:
scores = {}

In [280]:
if os.path.exists('final_setup/noseg_plo_scores.pkl'):
    scores = pickle.load(open('final_setup/noseg_plo_scores.pkl', 'rb'))

In [307]:
for file in os.scandir(output_folder):
    if file.name=='.ipynb_checkpoints':
        continue
    gold_name, inp, arch, w_embed, seed_num = file.name.split('.')[:-1]
    if (gold_name, inp, arch, w_embed, seed_num) not in scores:
        if len(gold_name.split('_'))>2:
            unit, pred_set, _ = gold_name.split('_')
            gold_path = decode_sets[unit][unit+'_'+pred_set+'_gold']
        else:
            unit, pred_set = gold_name.split('_')
            gold_path = decode_sets[unit][unit+'_'+pred_set]
        p, r, f = nem.evaluate_files(gold_path, file)
        scores[(gold_name, inp, arch, w_embed, seed_num)] = (p, r, f)
    

In [308]:
import pickle
pickle.dump(scores, open('final_setup/noseg_plo_scores.pkl', 'wb'))

In [309]:
score_tups = [(*k, *v) for k,v in scores.items()]

In [310]:
mev = pd.DataFrame(score_tups, columns=('gold_name', 'unit', 'arch', 
                                        'w_embed', 'seed_num', 
                                        'p_m', 'r_m', 'f_m'))
(mev[mev.gold_name.str.contains('dev')].groupby(['gold_name', 'unit', 'arch'])
 .f_m.agg(['max', 'min', 'mean', 'std', 'median', perc(95)]))

,,,max,min,mean,std,median,perc_95
gold_name,unit,arch,,,,,,
morph_dev_gold,morph,char_cnn,0.842105,0.801275,0.822758,0.009799,0.823080,0.838946
morph_dev_yap,morph,char_cnn,0.767338,0.722944,0.746338,0.011609,0.743981,0.765579
token_dev,token,char_cnn,0.813043,0.797814,0.804954,0.006030,0.804637,0.812646


In [311]:
(mev[mev.gold_name.str.contains('dev')].groupby(['gold_name', 'unit', 'arch']).size())

gold_name       unit   arch    
morph_dev_gold  morph  char_cnn    20
morph_dev_yap   morph  char_cnn    20
token_dev       token  char_cnn    10
dtype: int64

In [312]:
mev.head()

,gold_name,unit,arch,w_embed,seed_num,p_m,r_m,f_m
0,token_dev,token,char_cnn,ft_oov_tok,45_seed,0.836689,0.790698,0.813043
1,token_test,token,char_cnn,ft_oov_tok,45_seed,0.803965,0.791757,0.797814
2,token_dev,token,char_cnn,ft_oov_tok,44_seed,0.825792,0.771670,0.797814
3,token_test,token,char_cnn,ft_oov_tok,44_seed,0.795380,0.784165,0.789732
4,morph_dev_gold,morph,char_cnn,ft_oov_tok,46_seed,0.845455,0.786469,0.814896


In [313]:
mev['pred_set'] = mev.gold_name.apply(lambda x: '_'.join(x.split('_')[1:]))

In [314]:
mev = mev.merge(erdf, how='left')

In [315]:
(mev[mev.pred_set.str.contains('dev')].groupby(['unit', 'pred_set', 'arch', 'embed_type', 'cm'])
 .f_m.agg(['mean', 'std']).mul(100).round(2)
 .assign(mean = lambda x: x['mean'].apply('{:,.2f}'.format).astype(str)+' ± '+ x['std'].round(1).astype(str))[['mean']]
 .unstack([-2,-1]))


mean             
embed_type                    ft_oov             
cm                             Clash        Match
unit  pred_set arch                              
morph dev_gold char_cnn  81.59 ± 0.7  82.96 ± 0.7
      dev_yap  char_cnn  73.82 ± 0.8  75.44 ± 0.9
token dev      char_cnn          NaN  80.50 ± 0.6

In [316]:
x = (mev[mev.pred_set.str.contains('dev')].groupby(['unit', 'pred_set', 'arch', 'embed_type', 'cm'])
 .f_m.agg([ 'mean', 'std']).mul(100).round(2)
 .assign(std = lambda x: x['std'].round(1))
 .unstack([-2,-1]))
x.columns = x.columns.reorder_levels([1,2,0])
pd.set_option("max_columns", 30)
x.sort_index(axis=1)


embed_type              ft_oov                 
cm                       Clash       Match     
                          mean  std   mean  std
unit  pred_set arch                            
morph dev_gold char_cnn  81.59  0.7  82.96  0.7
      dev_yap  char_cnn  73.82  0.8  75.44  0.9
token dev      char_cnn    NaN  NaN  80.50  0.6

In [317]:
mev[(mev.unit=='morph') & (mev.pred_set.str.contains('pruned')) & (mev.embed_type=='ft_oov') & (mev.arch=='char_cnn')].groupby(['pred_set','cm']).f_m.mean().unstack()

cm
pred_set


In [318]:
mev['pred_set_sub'] = mev.pred_set.apply(lambda x: x.split('_')[1] if '_' in x else '')
mev['pred_set_main'] = mev.pred_set.apply(lambda x: x.split('_')[0] )
(mev[((mev.unit!='morph') & (mev.embed_type=='ft_oov') 
    ) 
    |
     ((mev.unit=='morph') 
       & (mev.embed_type=='ft_oov') 
      & (mev.arch=='char_cnn'))].groupby(['unit', 'pred_set_sub', 'cm', 'pred_set_main',])
 .f_m.mean().unstack().mul(100).round(2)
 .assign(ratio = lambda x: (x.test/x.dev -1).mul(100).round(1)))

pred_set_main               dev   test  ratio
unit  pred_set_sub cm                        
morph gold         Clash  81.59  78.86   -3.3
                   Match  82.96  79.76   -3.9
      yap          Clash  73.82  68.80   -6.8
                   Match  75.44  69.49   -7.9
token              Match  80.50  79.60   -1.1

In [260]:
mev.to_pickle('final_setup/noseg_plo_mev.pkl')

In [80]:
import os
from collections import defaultdict
for d in os.scandir('hp_search'):
    if d.name.startswith('models'):
        all_models_paths = defaultdict(list)
        all_models_epoch = defaultdict(lambda: -1)
        for f in os.scandir(d.path):
            if f.name!='.model' and f.name.endswith('.model'):
                a, c, _, e, _ = f.name.split('.')
                e = int(e)
                all_models_epoch[(a,c)] = max(alL_models_epoch[(a,c)], e)
                all_models_paths[(a,c)].append((e, f.path))
        for k, v in all_models_paths.items():
            for e, path in v:
                if e!=all_models_epoch[k]:
                    #os.remove(path)

        
